<a href="https://colab.research.google.com/github/Andrea-Graziano-Gitto/Andrea-Graziano-Gitto/blob/main/CVCreator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF Resume Generator – Required Setup Instructions

To properly use this PDF CV generator on Google Colab, you must prepare your fonts and configure the notebook correctly. Follow these instructions carefully.

--------------------------------------------------------------------------------

1. Prepare the `fonts` folder in your Google Drive

Create a folder in your Google Drive with this exact path:

    /MyDrive/fonts

Inside this folder, place the `.ttf` font files you want to use (e.g., Montserrat).
You MUST rename the files using the following naming convention:

Your Drive folder should contain:

    /MyDrive/fonts/
     ├── Montserrat-Regular.ttf
     ├── Montserrat-Bold.ttf
     ├── Montserrat-Italic.ttf
     └── Montserrat-BoldItalic.ttf

If you're using another font, such as Roboto, rename the files similarly:

    Roboto-Regular.ttf, Roboto-Bold.ttf, etc.

--------------------------------------------------------------------------------

2. Customize the font name in the code

In your Colab notebook, find this line:

    font_name = "Montserrat"  # <-- Change this if you're using a different font

Example: if you're using the "Roboto" font, write:

    font_name = "Roboto"

The rest of the code will automatically adapt to this name and search for the correct files.
Change font_name = "Montserrat" with your font name

--------------------------------------------------------------------------------

In [105]:
# Install libraries (if not already installed)
!pip install reportlab pillow

# === Import libraries ===
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.lib.colors import blue, black
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from PIL import Image
from google.colab import files
from google.colab import drive
import os

# === Mount Google Drive ===
drive.mount('/content/drive', force_remount=True)

# === Font settings ===
font_name = "Montserrat"
font_dir = "/content/drive/MyDrive/fonts"

# Variants with corresponding file extensions
variants = {
    "regular": f"{font_name}-Regular.ttf",
    "bold": f"{font_name}-Bold.ttf",
    "italic": f"{font_name}-Italic.ttf",
    "bolditalic": f"{font_name}-BoldItalic.ttf"
}

# Map exact font names to register (for pdfmetrics.registerFont)
variant_name_map = {
    "regular": f"{font_name}-Regular",
    "bold": f"{font_name}-Bold",
    "italic": f"{font_name}-Italic",
    "bolditalic": f"{font_name}-BoldItalic"
}

# Register all font variants
for variant_key, variant_file in variants.items():
    variant_path = os.path.join(font_dir, variant_file)
    if os.path.exists(variant_path):
        registered_font_name = variant_name_map[variant_key]
        pdfmetrics.registerFont(TTFont(registered_font_name, variant_path))
        print(f"✅ Font variant '{registered_font_name}' loaded from {variant_path}")
    else:
        print(f"⚠️ Font variant not found: {variant_path}")

# List with exact font names to use in code
variant = [
    variant_name_map["regular"],     # variant[0]
    variant_name_map["bold"],        # variant[1]
    variant_name_map["italic"],      # variant[2]
    variant_name_map["bolditalic"]   # variant[3]
]

print("Variant list:", variant)


Mounted at /content/drive
✅ Font variante 'Montserrat-Regular' caricato da /content/drive/MyDrive/fonts/Montserrat-Regular.ttf
✅ Font variante 'Montserrat-Bold' caricato da /content/drive/MyDrive/fonts/Montserrat-Bold.ttf
✅ Font variante 'Montserrat-Italic' caricato da /content/drive/MyDrive/fonts/Montserrat-Italic.ttf
✅ Font variante 'Montserrat-BoldItalic' caricato da /content/drive/MyDrive/fonts/Montserrat-BoldItalic.ttf
Lista variant: ['Montserrat-Regular', 'Montserrat-Bold', 'Montserrat-Italic', 'Montserrat-BoldItalic']


In [106]:
def add_image(c, multiply=True, multiplier=1.0, right=True, top=True, margin_x=1.3, margin_y=1.3):
    print("👉 Please upload a PNG or JPG image:")
    uploaded = files.upload()

    if not uploaded:
        print("⚠️ No file uploaded.")
        return

    filename = next(iter(uploaded))

    # If the file exists, delete it before overwriting
    if os.path.exists(filename):
        try:
            os.remove(filename)
            print(f"🗑️ Previous file '{filename}' deleted.")
        except Exception as e:
            print(f"❌ Error deleting existing file: {e}")
            return

    # Save the uploaded file to disk
    with open(filename, 'wb') as f:
        f.write(uploaded[filename])
    print(f"📥 File '{filename}' saved to disk.")

    # (Optional) cleanup of similar temporary files if needed
    # e.g., files named like filename + "_temp", etc.

    # Proceed to open image and insert into PDF
    try:
        im = Image.open(filename)
        width_px, height_px = im.size
        print(f"📐 Original image size: {width_px}px x {height_px}px")

        px_to_cm = 0.02646
        scale = multiplier if multiply else (1.0 / multiplier)
        width_cm = width_px * px_to_cm * scale
        height_cm = height_px * px_to_cm * scale

        page_width_pt, page_height_pt = c._pagesize
        pt_to_cm = 2.54 / 72
        page_width_cm = page_width_pt * pt_to_cm
        page_height_cm = page_height_pt * pt_to_cm

        x_cm = page_width_cm - width_cm - margin_x if right else margin_x
        y_cm = page_height_cm - height_cm - margin_y if top else margin_y

        c.drawImage(filename, x_cm * cm, y_cm * cm, width=width_cm * cm, height=height_cm * cm)
        print(f"✅ Image inserted at scale {scale:.2f}x at ({x_cm:.2f}cm, {y_cm:.2f}cm)")

    except Exception as e:
        print(f"❌ Error inserting image: {e}")


In [107]:
from reportlab.lib.units import cm
from reportlab.lib.colors import blue, black

left_margin_cm = 2.5
indent_paragraph_cm = 1.0

def parse_segments(text):
    import re
    # Search for \b...\b, \i...\i, \h...\h formatting tags
    pattern = r'(\\b|\\i|\\h)(.+?)\1'
    segments = []
    last = 0
    for match in re.finditer(pattern, text):
        start, end = match.span()
        if start > last:
            # Add normal text before the tag
            segments.append((text[last:start], 'normal'))
        tag = match.group(1)
        content = match.group(2)
        style = {'\\b': 'bold', '\\i': 'italic', '\\h': 'hyperlink'}[tag]
        segments.append((content, style))
        last = end
    # Add remaining normal text after last tag
    if last < len(text):
        segments.append((text[last:], 'normal'))
    return segments


def write_generic_text(c, font_size, text, y_pos_cm, variant, indent=False, link=None, space_after_block_cm=None):
    x = (left_margin_cm + (indent_paragraph_cm if indent else 0)) * cm
    y = y_pos_cm * cm
    line_spacing = font_size + 2

    # Normalize link parameter to a list
    if link is None:
        link_list = []
    elif isinstance(link, str):
        link_list = [link]
    else:
        link_list = link

    link_index = 0  # index for links in the list

    lines = text.split('\n')
    for i, line in enumerate(lines):
        y_line = y - i * line_spacing
        x_cursor = x
        for segment, style in parse_segments(line):
            if style == 'bold':
                c.setFont(variant[1], font_size)
                c.setFillColor(black)
            elif style == 'italic':
                c.setFont(variant[2], font_size)
                c.setFillColor(black)
            elif style == 'hyperlink':
                if link_index < len(link_list):
                    current_link = link_list[link_index]
                    link_index += 1
                    c.setFont(variant[0], font_size)
                    c.setFillColor(blue)
                    text_width = c.stringWidth(segment, variant[0], font_size)
                    c.drawString(x_cursor, y_line, segment)
                    c.linkURL(current_link, (x_cursor, y_line - 2, x_cursor + text_width, y_line + font_size), relative=0)
                    x_cursor += text_width
                    continue
                else:
                    # No more links available, draw as normal black text
                    c.setFont(variant[0], font_size)
                    c.setFillColor(black)
            else:  # normal text
                c.setFont(variant[0], font_size)
                c.setFillColor(black)

            c.drawString(x_cursor, y_line, segment)
            text_width = c.stringWidth(segment, c._fontname, font_size)
            x_cursor += text_width

    # Default space after block if not set or empty string
    if space_after_block_cm is None or (isinstance(space_after_block_cm, str) and space_after_block_cm.strip() == ""):
        space_after_block_cm = 0.3

    total_height_cm = len(lines) * line_spacing / cm + float(space_after_block_cm)
    return y_pos_cm - total_height_cm


In [108]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm

##########################################################################################
# ↓↓↓ scrivi_testo_generico: How to Use This Function to Write Formatted Text in PDF ↓↓↓ #
##########################################################################################
#
# This function writes text to the ReportLab canvas `c`, supporting:
# - Font variants (bold, italic, normal)
# - Line breaks (\n)
# - Paragraph indentation
# - Clickable hyperlinks (with optional multiple links)
#
# === REQUIRED PARAMETERS ===
# - c              : ReportLab canvas object
# - font_size      : Font size in points (e.g., 12)
# - testo          : String to draw, may contain formatting tags:
#                    • \b...\b → bold text
#                    • \i...\i → italic text
#                    • \h...\h → hyperlink text (blue, underlined)
# - y_pos_cm       : Vertical position in cm from the bottom of the page
# - variant        : List of registered font names in order:
#                      [regular, bold, italic, bolditalic]
#
# === OPTIONAL PARAMETERS ===
# - indentato             : True to apply paragraph indentation (default: False)
# - link                  : A single URL (for one \h...\h), or a list of URLs (multiple \h...\h)
# - spazio_tra_blocchi_cm : Vertical spacing after the block (default: 0.3 cm)
#
# === RETURN VALUE ===
# - Returns the Y coordinate (in cm) for the next available vertical position
#
# === EXAMPLE USAGE ===
# scrivi_testo_generico(
#     c=my_canvas,
#     font_size=12,
#     testo="This is \bimportant\b and this is \iemphasized\i.\nVisit \hour site\h!",
#     y_pos_cm=25,
#     variant=["Montserrat-Regular", "Montserrat-Bold", "Montserrat-Italic", "Montserrat-BoldItalic"],
#     indentato=True,
#     link="https://example.com"
# )
#
##########################################################################################

##########################################################################################
# ↓↓↓ aggiungiImg: How to Upload and Insert an Image into a PDF using ReportLab ↓↓↓     #
##########################################################################################
#
# This function allows uploading a PNG or JPG image in Google Colab via `files.upload()`,
# deletes any existing image with the same name, resizes, and inserts it into a PDF canvas.
#
# === REQUIRED PARAMETER ===
# - c : ReportLab canvas object (e.g. from `canvas.Canvas(...)`)
#
# === OPTIONAL PARAMETERS ===
# - moltiplica       : If True, scales image *by* `moltiplicatore`, else scales /by/
# - moltiplicatore   : Scaling factor (e.g., 0.5 = half-size, 2.0 = double-size)
# - destra           : If True, aligns image to the right margin (minus `menox`), else to left
# - sopra            : If True, aligns image to the top margin (minus `menoy`), else to bottom
# - menox / menoy    : Margins in cm from the edge (horizontal or vertical)
#
# === PLACEMENT LOGIC ===
# • Image is resized using DPI to cm: 1 px ≈ 0.02646 cm
# • Page size is detected from canvas (supports any page size, not just A4)
# • Final position is calculated based on page width/height and margins
#
# === USAGE EXAMPLE ===
# from reportlab.pdfgen import canvas
# from reportlab.lib.pagesizes import A4
#
# c = canvas.Canvas("output.pdf", pagesize=A4)
# aggiungiImg(c, moltiplicatore=0.7, destra=True, sopra=True, menox=2, menoy=3)
# c.save()
#
# === INTERACTIVE PROMPT ===
# 🖼️ User will see: "👉 Upload a PNG or JPG image:"
#     → Upload via the dialog to insert into the PDF
#
##########################################################################################

##########################################################################################
# ⏯️ FINAL NOTE: Managing PDF Generation in Google Colab                                #
##########################################################################################
#
# ✅ To generate the PDF, click "▶ Run All" or run each cell manually in order
#
# 📝 To add pages:
#     - Call `c.showPage()` to start a new page
#     - Reset vertical position using:
#         y_cm = (page_height / cm) - <top_margin_in_cm>
#
# 📄 Every `c.showPage()` creates a new page.
#     Call `c.save()` once at the end to finalize and close the document.
#
# 📐 To control spacing:
#     - Decrease `y_cm` to move downward
#     - Increase `spazio_tra_blocchi_cm` to add spacing between text blocks
#
# 🖼️ To insert more images, call `aggiungiImg(c, ...)` again
#
# 🚀 After everything is complete, download the final PDF from the Colab file browser.
#
##########################################################################################

# ==== BASIC INITIALIZATION EXAMPLE ====

name = "TiTle"
output_file = name+".pdf"
page_width, page_height = A4

# Create the canvas PDF
c = canvas.Canvas(output_file, pagesize=A4)
c.setTitle(name)

# Initial vertical position from top margin (2 cm)
y_cm = (page_height / cm) - 2

# Save and close the page
c.showPage()
c.save()

print(f"✅ PDF saved as: {output_file}")


👉 Carica un'immagine PNG o JPG:


Saving testa2.png to testa2 (3).png
🗑️ File precedente 'testa2 (3).png' eliminato.
📥 File 'testa2 (3).png' salvato su disco.
📐 Dimensione immagine originale: 901px x 901px
✅ Immagine inserita con scala 0.25x a (13.74cm, 22.44cm)
✅ PDF salvato come: documento_finale.pdf
